# Speech to text with Speech Recognition

In [1]:
import sys

#Install fuzzywuzzy for keyword match
!{sys.executable} -m pip install fuzzywuzzy

#intall pyaudio for access to local microphone
!{sys.executable} -m pip install PyAudio

#Install pydub for silence detection and for handling Audio files.
!{sys.executable} -m pip install pydub

#Install a pip package in the current Jupyter kernel
!{sys.executable} -m pip install SpeechRecognition

#Install a pip package in the current Jupyter kernel
!{sys.executable} -m pip install google-cloud-storage

 #Install a pip package in the current Jupyter kernel
!{sys.executable} -m pip install --upgrade google-cloud-speech

Requirement already up-to-date: google-cloud-speech in ./ispeech/lib/python3.7/site-packages (1.3.2)


# Get the Audio Book and Save the file

In [2]:
import requests

## These audio_books are in the public domain
url = 'http://www.archive.org/download/alices_adventures/alices_adventures_64kb_mp3.zip'
r = requests.get(url, allow_redirects=True)

## we save the zip_file in our directory
open('data/Aliceinwonderland.zip', 'wb').write(r.content)

28153636

In [3]:
## Extracting the zip_files

# importing required modules 
from zipfile import ZipFile 

# specifying the zip file name 
file_name = "data/Aliceinwonderland.zip"

# opening the zip file in READ mode 
with ZipFile(file_name, 'r') as zipfile: 
    # printing all the contents of the zip file 
    zipfile.printdir() 

    # extracting all the files 
    print('Extracting all the files now...') 
    zipfile.extractall(path = 'data/Alice/') 
    print('Done!') 


File Name                                             Modified             Size
aliceinwonderland_01_carroll_64kb.mp3          2007-05-19 00:31:06      3538287
aliceinwonderland_02_carroll_64kb.mp3          2007-05-19 00:32:44      3086679
aliceinwonderland_03_carroll_64kb.mp3          2007-05-19 00:32:30      3286894
aliceinwonderland_04_carroll_64kb.mp3          2007-05-19 00:32:08      3569020
aliceinwonderland_05_carroll_64kb.mp3          2007-05-19 00:31:20      3142903
aliceinwonderland_06_carroll_64kb.mp3          2007-05-19 00:31:32      2640087
aliceinwonderland_07_carroll_64kb.mp3          2007-05-19 00:31:54      1746909
aliceinwonderland_08_carroll_64kb.mp3          2007-05-19 00:31:46      3518858
aliceinwonderland_09_carroll_64kb.mp3          2007-05-19 00:32:16      1794561
aliceinwonderland_10_carroll_64kb.mp3          2007-05-19 00:32:52      1827576
Extracting all the files now...
Done!


# Audio data with Python


# Dividing a chapter into chunks and indexing

In this section we will use SpeechRecognition and Pydub packages only

In [ ]:
import speech_recognition as  sr 
import  os
from pydub import AudioSegment
from  pydub.silence  import  split_on_silence
from pydub.silence import detect_nonsilent
from pydub.playback import play
def silence_based_conversion(path = ""): 

    # open the audio file stored in 
    # the local system as a wav file. 
    song = AudioSegment.from_mp3(path)

    # open a file where we will concatenate 
    # and store the recognized text 
    fh = open("recognized.txt", "w+") 
    new_list = []
    # split track where silence is 0.5 seconds 
    # or more and get chunks 
    chunks = split_on_silence(song, 
        # must be silent for at least 0.5 seconds 
        # or 500 ms. adjust this value based on user 
        # requirement. if the speaker stays silent for 
        # longer, increase this value. else, decrease it. 
        min_silence_len = 750, 

        # consider it silent if quieter than -16 dBFS 
        # adjust this per requirement 
        silence_thresh = -40, 
        keep_silence= 0
    ) 

    ranges = detect_nonsilent(song, min_silence_len= 750, silence_thresh= -40)
    # create a directory to store the audio chunks. 
    try: 
        os.mkdir('audio_chunks') 
    except(FileExistsError): 
        pass

    # move into the directory to 
    # store the audio files. 
    os.chdir('audio_chunks') 

    i = 0
    # process each chunk 
    start = 0
    for chunk, rng in zip(chunks, ranges):	
        
        # Create 0.5 seconds silence chunk 
        chunk_silent = AudioSegment.silent(duration = 300) 

        # add 0.5 sec silence to beginning and 
        # end of audio chunk. This is done so that 
        # it doesn't seem abruptly sliced. 
        audio_chunk = chunk_silent + chunk + chunk_silent 

        # export audio chunk and save it in 
        # the current directory. 
        print("saving chunk{0}.wav".format(i)) 
        # specify the bitrate to be 192 k 
        audio_chunk.export("./chunk{0}.wav".format(i), bitrate ='192k', format ="wav") 

        # the name of the newly created chunk 
        filename = 'chunk'+str(i)+'.wav'

#         print("Processing chunk "+str(i)) 

        # get the name of the newly created chunk 
        # in the AUDIO_FILE variable for later use. 
        file = filename 

        # create a speech recognition object 
        r = sr.Recognizer() 

        # recognize the chunk 
        with sr.AudioFile(file) as source: 
            # remove this if it is not working 
            # correctly. 
            # r.adjust_for_ambient_noise(source) 
            audio_listened = r.listen(source)

        try: 
            # try converting it to text 
            rec = r.recognize_google(audio_listened) 
            # write the output to the file. 
            fh.write(rec+".")
            new_list.append((i,rng, rec))

        # catch any errors. 
        except sr.UnknownValueError: 
            print("Could not understand audio") 

        except sr.RequestError as e: 
            print("Could not request results. check your internet connection") 

        i += 1
        start += len(chunk) + 1000

    os.chdir('/Users/muratguner/Documents/experiments/meet_up_presentation') 
    
    return new_list
#/Users/muratguner/Documents/experiments/meet_up_presentation/data/Alice/aliceinwonderland_01_carroll_64kb.mp3
if __name__ == '__main__': 

    print('Enter the audio file path') 

    path = input() 

    text = silence_based_conversion(path) 

# Keyword match with FuzzyWuzzy

In [55]:
from fuzzywuzzy import fuzz

for (i, interval,sentence) in text:
    score = fuzz.token_set_ratio('long hall', sentence)
    if score > 70: 
        print(i, score, sentence)
        song = AudioSegment.from_mp3("/Users/muratguner/Documents/experiments/meet_up_presentation/data/Alice/aliceinwonderland_01_carroll_64kb.mp3")
        play(song[interval[0]:interval[1]])

24 100 she found herself in a long low Hall which was lit up by a row of lamps hanging from the roof


# Taking input keyword from the microphone

In [79]:
import random
import time

from fuzzywuzzy import fuzz


import speech_recognition as sr


def recognize_speech_from_mic(recognizer, microphone):
    """Transcribe speech from recorded from `microphone`.
    Returns a dictionary with three keys:
    "success": a boolean indicating whether or not the API request was
               successful
    "error":   `None` if no error occured, otherwise a string containing
               an error message if the API could not be reached or
               speech was unrecognizable
    "transcription": `None` if speech could not be transcribed,
               otherwise a string containing the transcribed text
    """
    # check that recognizer and microphone arguments are appropriate type
    if not isinstance(recognizer, sr.Recognizer):
        raise TypeError("`recognizer` must be `Recognizer` instance")

    if not isinstance(microphone, sr.Microphone):
        raise TypeError("`microphone` must be `Microphone` instance")

    # adjust the recognizer sensitivity to ambient noise and record audio
    # from the microphone
    with microphone as source:
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)

    # set up the response object
    response = {
        "success": True,
        "error": None,
        "transcription": None
    }

    # try recognizing the speech in the recording
    # if a RequestError or UnknownValueError exception is caught,
    #     update the response object accordingly
    try:
        response["transcription"] = recognizer.recognize_google(audio)
    except sr.RequestError:
        # API was unreachable or unresponsive
        response["success"] = False
        response["error"] = "API unavailable"
    except sr.UnknownValueError:
        # speech was unintelligible
        response["error"] = "Unable to recognize speech"

    return response


if __name__ == "__main__":
    # create recognizer and mic instances
    recognizer = sr.Recognizer()
    microphone = sr.Microphone()
    PROMPT_LIMIT = 5
    # format the instructions string
    instructions = (
        "Give me a keyword to search in Alice in Wonderland"
    )

    # show instructions and wait 3 seconds before starting the game
    print(instructions)
    
    song = AudioSegment.from_mp3("/Users/muratguner/Documents/experiments/meet_up_presentation/data/Alice/aliceinwonderland_01_carroll_64kb.mp3")

    
    for j in range(PROMPT_LIMIT):
        print('Attempt_{} Speak!'.format(j))
        keyword = recognize_speech_from_mic(recognizer, microphone)
        if keyword["transcription"]:
            break
        if not keyword["success"]:
            break
        print("I didn't catch that. What did you say?\n")  
    print(keyword['transcription'])
    for (i, interval,sentence) in text:
        score = fuzz.token_set_ratio(keyword['transcription'], sentence)
        if score > 95: 
            print(i, score, sentence)
            play(song[interval[0]:interval[1]])
            print('Do you want to hear next? Say next or continue')
            gate = recognize_speech_from_mic(recognizer, microphone)
            if gate['transcription'].lower() == 'next':
                print('You said: {}'.format(gate['transcription']))
                print(gate['success'])
                continue
            elif gate['transcription'].lower() == 'continue':
                print(gate['transcription'])
                play(song[interval[1]-5*1000: interval[1]+15*1000])
                break


Give me a keyword to search in Alice in Wonderland
Attempt_0 Speak!
I didn't catch that. What did you say?

Attempt_1 Speak!
down
8 100 down the rabbit
Do you want to hear next? Say next or continue
You said: next
True
13 100 but when the rabbit actually took a watch out of its waistcoat pocket and looked at it and then hurried on Alice started to her feet or it flashed across her mind that she had never before seen a rabbit with either a waistcoat pocket or watch to take out of it and burning with curiosity she ran across the field after it and was just in time to see it pop down a large rabbit hole under the Hedge in another moment down went Alice after
Do you want to hear next? Say next or continue
You said: next
True
14 100 the rabbit hole with straight on like a tunnel for some way and then dip suddenly down there suddenly that Alice had not a moment to think about stopping herself before she found herself falling down what seemed to be a very deep well
Do you want to hear next? S

AttributeError: 'NoneType' object has no attribute 'lower'

# Resources:

[Speech Recognition](https://pypi.org/project/SpeechRecognition/)

[For taking the input from microphone](https://github.com/realpython/python-speech-recognition/blob/master/guessing_game.py)

[For the silence detection](https://github.com/jiaaro/pydub/blob/master/pydub/silence.py)

[For keyword matching with the text](https://pypi.org/project/fuzzywuzzy/)

[Alice in Wonderland Text](https://www.gutenberg.org/files/11/11-h/11-h.htm#chap01)

[Alice in Wonderland Audiobook](https://archive.org/details/alices_adventures)

# Extras

## Detecting Silence and Non_silence

In [5]:
from pydub.silence  import  detect_silence
from pydub.silence  import detect_nonsilent
path = "/Users/muratguner/Documents/experiments/meet_up_presentation/data/Alice/aliceinwonderland_01_carroll_64kb.mp3"
song = AudioSegment.from_mp3(path)


nonsilence_range = detect_nonsilent(song[:30*1000], min_silence_len=750, silence_thresh= -40)

splitted_audio = split_on_silence(audio_segment=song[:30*1000], min_silence_len= 750, keep_silence= 300, silence_thresh= -40)



## Getting Timestamps with Google_Cloud_Speech_Recognition

A: Dividing the audio files into 30 seconds pieces
    

In [236]:
from pydub import AudioSegment

# Audio = AudioSegment.from_mp3("/Users/muratguner/Documents/experiments/google_cloud_speech/resources/alices_adventures_64kb_mp3/aliceinwonderland_03_carroll_64kb.mp3")
Audio = AudioSegment.from_mp3("/Users/muratguner/Documents/experiments/google_cloud_speech/resources/alices_adventures_64kb_mp3/aliceinwonderland_01_carroll_64kb.mp3")

os.chdir("/Users/muratguner/Documents/experiments/meet_up_presentation/")

try: 
    os.mkdir('data/parts_of_chapter_1') 
except(FileExistsError): 
    pass
os.chdir('data/parts_of_chapter_1') 

i = 0

for t1 in range(0, len(Audio), 30*1000):
    new_audio = Audio[t1: t1+30*1000]
    new_audio.export("./chunk{0}.wav".format(i), bitrate ='192k', format ="wav")
    i+=1
os.chdir("/Users/muratguner/Documents/experiments/meet_up_presentation/")

Chapter 1 divided into 30 seconds pieces and recorded at data/parts_of_chapter_1

In [240]:
from google.cloud import speech_v1
from google.oauth2 import service_account


def sample_long_running_recognize(storage_uri):
    """
    Print start and end time of each word spoken in audio file from Cloud Storage

    Args:
      storage_uri URI for audio file in Cloud Storage, e.g. gs://[BUCKET]/[FILE]
    """
    
    credentials = service_account.Credentials.from_service_account_file('google_credentials.json')


    client = speech_v1.SpeechClient(credentials=credentials)

    # storage_uri = 'gs://cloud-samples-data/speech/brooklyn_bridge.flac'

    # When enabled, the first result returned by the API will include a list
    # of words and the start and end time offsets (timestamps) for those words.
    enable_word_time_offsets = True

    # The language of the supplied audio
    language_code = "en-US"
    config = {
        "enable_word_time_offsets": enable_word_time_offsets,
        "language_code": language_code,
    }
#   audio = {"uri": storage_uri}

    with io.open(storage_uri, "rb") as f:
        content = f.read()
    audio = {"content": content}


    operation = client.long_running_recognize(config, audio)

    print(u"Waiting for operation to complete...")
    response = operation.result()
#     print(response)
    # The first result includes start and end time word offsets
    result = response.results[0]
    # First alternative is the most probable result
    alternative = result.alternatives[0]
    print(alternative.transcript, alternative.words[1].start_time.seconds ,alternative.words[-1].end_time.seconds)
#     print(u"Transcript: {}".format(alternative.transcript))
    # Print the start and end time of each word
#     for word in alternative.words.:
#         print(u"Word: {}".format(word.word))
# #         print(
# #             u"Start time: {} seconds {} nanos".format(
# #                 word.start_time.seconds, word.start_time.nanos
# #             )
# #         )
# #         print(
# #             u"End time: {} seconds {} nanos".format(
# #                 word.end_time.seconds, word.end_time.nanos
# #             )
# #         )



In [ ]:
local_file_path = "/Users/muratguner/Documents/experiments/meet_up_presentation/data/parts_of_chapter_1/chunk5.wav"
sample_long_running_recognize(local_file_path)


With speech_v1 and content

In [264]:
from google.cloud import speech_v1

def sample_recognize(content):
    """
    Transcribe a short audio file using synchronous speech recognition

    Args:
      local_file_path Path to local audio file, e.g. /path/audio.wav
    """
    credentials = service_account.Credentials.from_service_account_file('google_credentials.json')

    client = speech_v1.SpeechClient(credentials = credentials)

    # local_file_path = 'resources/brooklyn_bridge.raw'

    # The language of the supplied audio
    language_code = "en-US"

    # Sample rate in Hertz of the audio data sent
    sample_rate_hertz = 22000

    # Encoding of audio data sent. This sample sets this explicitly.
    # This field is optional for FLAC and WAV audio formats.
    encoding = enums.RecognitionConfig.AudioEncoding.LINEAR16
    config = {
        "language_code": language_code,
        "sample_rate_hertz": sample_rate_hertz,
        "encoding": encoding,
        "enable_word_time_offsets": True
    }
    audio = {"content": content}

    response = client.recognize(config, audio)
    i = 0
    for result in response.results:
        # First alternative is the most probable result
        alternative = result.alternatives[0]
        print(t1,u"Transcript: {}{}".format(alternative.transcript, i))
        for word in alternative.words:
            print(u"Word: {}".format(word.word))
            print(
            u"Start time: {} seconds {} nanos".format(
                word.start_time.seconds, word.start_time.nanos
                )
            )
            print(
              u"End time: {} seconds {} nanos".format(
                  word.end_time.seconds, word.end_time.nanos
                  )
              )

        i+=1

In [268]:
Audio = AudioSegment.from_mp3(local_file_path)



type(Audio.)

bytes

Waiting for operation to complete...
there was nothing else to do so Alliston began talking to herself Dino will miss me very much tonight I should think. It was the cat I hope they'll remember her saucer of milk at tea-time Dynamite deer wish you were down here with me Alice felt that she was dozing off when suddenly dumped down She Came Upon a heap of sticks and dry leaves and the phone is over 0 22


## Playing corresponding Chunk

In [168]:
## Let's play the audio file first

from pydub.playback import play
from pydub import AudioSegment

song = AudioSegment.from_mp3(local_file_path)

play(song[: 10*1000])

## This is our first Transcription

In [140]:
## Let's play the audio file first

from pydub.playback import play
from pydub import AudioSegment

song = AudioSegment.from_mp3("data/Alice/aliceinwonderland_01_carroll_64kb.mp3")

play(song[: 10*1000])

In [18]:
from google.cloud import speech_v1
from google.cloud.speech_v1 import enums
import io


def sample_recognize(local_file_path):
    """
    Transcribe a short audio file using synchronous speech recognition

    Args:
      local_file_path Path to local audio file, e.g. /path/audio.wav
    """
    credentials = service_account.Credentials. from_service_account_file('google_credentials.json')

    client = speech_v1p1beta1.SpeechClient(credentials= credentials)

    # local_file_path = 'resources/brooklyn_bridge.raw'

    # The language of the supplied audio
    language_code = "en-US"

    # Sample rate in Hertz of the audio data sent
    sample_rate_hertz = 22050

    # Encoding of audio data sent. This sample sets this explicitly.
    # This field is optional for FLAC and WAV audio formats.
    encoding = enums.RecognitionConfig.AudioEncoding.LINEAR16
    config = {
        "language_code": language_code,
        "sample_rate_hertz": sample_rate_hertz,
        "encoding": encoding,
    }
    with io.open(local_file_path, "rb") as f:
        content = f.read()
    audio = {"content": content}

    response = client.recognize(config, audio)
    for result in response.results:
        # First alternative is the most probable result
        alternative = result.alternatives[0]
        print(u"Transcript: {}".format(alternative.transcript))


In [19]:
sample_recognize(local_file_path= "data/chunk7.wav")

Transcript: for a minute or two she stood looking at the house when suddenly a footman and Livery came running out of the wood judging by his face only she would have called him a fish and wrapped loudly at the door with his knuckles is opened by another freaking delivery with a round face and large eyes like a frog


In [150]:
from pydub import AudioSegment
from pydub.playback import play
# from transcribe_short_audio import sample_recognize
from google.cloud import speech_v1p1beta1
from google.cloud.speech_v1 import enums
import io
from pydub.playback import play

## We cannot just pass the json credentials

## For more details: https://github.com/googleapis/google-cloud-python/issues/5349
from google.oauth2 import service_account

In [153]:
def sample_recognize(content):
    """
    Transcribe a short audio file using synchronous speech recognition

    Args:
      local_file_path Path to local audio file, e.g. /path/audio.wav
    """
    credentials = service_account.Credentials. from_service_account_file('google_credentials.json')

    client = speech_v1p1beta1.SpeechClient(credentials= credentials)

    # local_file_path = 'resources/brooklyn_bridge.raw'

    # The language of the supplied audio
    language_code = "en-US"

    # Sample rate in Hertz of the audio data sent
    sample_rate_hertz = 22000

    # Encoding of audio data sent. This sample sets this explicitly.
    # This field is optional for FLAC and WAV audio formats.
    encoding = enums.RecognitionConfig.AudioEncoding.LINEAR16
    config = {
        "language_code": language_code,
        "sample_rate_hertz": sample_rate_hertz,
        "encoding": encoding,
        "enable_word_confidence": True,
        "enable_word_time_offsets": True
    }
    audio = {"content": content}

    response = client.recognize(config, audio)
    i = 0
    for result in response.results:
        # First alternative is the most probable result
        alternative = result.alternatives[0]
        print(t1,u"Transcript: {}{}".format(alternative.transcript, i))
        for word in alternative.words:
          print(u"Word: {}".format(word.word))
          print(u"Confidence: {}".format(word.confidence))
          print(
            u"Start time: {} seconds {} nanos".format(
                word.start_time.seconds, word.start_time.nanos
                )
            )
          print(
              u"End time: {} seconds {} nanos".format(
                  word.end_time.seconds, word.end_time.nanos
                  )
              )

        i+=1


# for t1 in range(1, 20, 5):
#     t2= t1+ 10
#     t1 = t1 * 1000 #Works in milliseconds
#     t2 = t2 * 1000
#     newAudio = Audio[t1:t2]

#     newAudio.export('newSong1.wav', format="wav") 
# play(Audio[0:15*1000])
t1=15
t2= t1+ 10
t1 = t1 * 1000 #Works in milliseconds
t2 = t2 * 1000

# Audio = AudioSegment.from_mp3("/Users/muratguner/Documents/experiments/google_cloud_speech/resources/alices_adventures_64kb_mp3/aliceinwonderland_03_carroll_64kb.mp3")
Audio = AudioSegment.from_mp3("/Users/muratguner/Documents/experiments/google_cloud_speech/resources/alices_adventures_64kb_mp3/aliceinwonderland_03_carroll_64kb.mp3")

newAudio = Audio[t1:t2]

sample_recognize(newAudio.raw_data)

15000 Transcript: recorded by Kirsten ferreri0
Word: recorded
Confidence: 1.0
Start time: 0 seconds 800000000 nanos
End time: 1 seconds 500000000 nanos
Word: by
Confidence: 1.0
Start time: 1 seconds 500000000 nanos
End time: 1 seconds 600000000 nanos
Word: Kirsten
Confidence: 0.8100687265396118
Start time: 1 seconds 600000000 nanos
End time: 2 seconds 100000000 nanos
Word: ferreri
Confidence: 0.6041839718818665
Start time: 2 seconds 100000000 nanos
End time: 2 seconds 600000000 nanos
15000 Transcript:  Alice's Adventures in Wonderland by Lewis Carroll chapter 3 a caucus race1
Word: Alice's
Confidence: 0.9876290559768677
Start time: 3 seconds 500000000 nanos
End time: 4 seconds 200000000 nanos
Word: Adventures
Confidence: 0.9876290559768677
Start time: 4 seconds 200000000 nanos
End time: 4 seconds 500000000 nanos
Word: in
Confidence: 0.9876290559768677
Start time: 4 seconds 500000000 nanos
End time: 4 seconds 700000000 nanos
Word: Wonderland
Confidence: 0.9876290559768677
Start time: 4 

In [152]:

def recognize_speech_from_mic(recognizer, microphone):
    """Transcribe speech from recorded from `microphone`.

    Returns a dictionary with three keys:
    "success": a boolean indicating whether or not the API request was
               successful
    "error":   `None` if no error occured, otherwise a string containing
               an error message if the API could not be reached or
               speech was unrecognizable
    "transcription": `None` if speech could not be transcribed,
               otherwise a string containing the transcribed text
    """
    # check that recognizer and microphone arguments are appropriate type
    if not isinstance(recognizer, sr.Recognizer):
        raise TypeError("`recognizer` must be `Recognizer` instance")

    if not isinstance(microphone, sr.Microphone):
        raise TypeError("`microphone` must be `Microphone` instance")

    # adjust the recognizer sensitivity to ambient noise and record audio
    # from the microphone
    with microphone as source:
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)

    # set up the response object
    response = {
        "success": True,
        "error": None,
        "transcription": None
    }

    # try recognizing the speech in the recording
    # if a RequestError or UnknownValueError exception is caught,
    #     update the response object accordingly
    try:
        response["transcription"] = recognizer.recognize_google(audio)
    except sr.RequestError:
        # API was unreachable or unresponsive
        response["success"] = False
        response["error"] = "API unavailable"
    except sr.UnknownValueError:
        # speech was unintelligible
        response["error"] = "Unable to recognize speech"

    return response


if __name__ == "__main__":
    # set the list of words, maxnumber of guesses, and prompt limit
    WORDS = ["apple", "banana", "grape", "orange", "mango", "lemon"]
    NUM_GUESSES = 3
    PROMPT_LIMIT = 5

    # create recognizer and mic instances
    recognizer = sr.Recognizer()
    microphone = sr.Microphone()

    # get a random word from the list
    word = random.choice(WORDS)

    # format the instructions string
    instructions = (
        "I'm thinking of one of these words:\n"
        "{words}\n"
        "You have {n} tries to guess which one.\n"
    ).format(words=', '.join(WORDS), n=NUM_GUESSES)

    # show instructions and wait 3 seconds before starting the game
    print(instructions)
    time.sleep(3)

    for i in range(NUM_GUESSES):
        # get the guess from the user
        # if a transcription is returned, break out of the loop and
        #     continue
        # if no transcription returned and API request failed, break
        #     loop and continue
        # if API request succeeded but no transcription was returned,
        #     re-prompt the user to say their guess again. Do this up
        #     to PROMPT_LIMIT times
        for j in range(PROMPT_LIMIT):
            print('Guess {}. Speak!'.format(i+1))
            guess = recognize_speech_from_mic(recognizer, microphone)
            if guess["transcription"]:
                break
            if not guess["success"]:
                break
            print("I didn't catch that. What did you say?\n")

        # if there was an error, stop the game
        if guess["error"]:
            print("ERROR: {}".format(guess["error"]))
            break

        # show the user the transcription
        print("You said: {}".format(guess["transcription"]))

        # determine if guess is correct and if any attempts remain
        guess_is_correct = guess["transcription"].lower() == word.lower()
        user_has_more_attempts = i < NUM_GUESSES - 1

        # determine if the user has won the game
        # if not, repeat the loop if user has more attempts
        # if no attempts left, the user loses the game
        if guess_is_correct:
            print("Correct! You win!".format(word))
            break
        elif user_has_more_attempts:
            print("Incorrect. Try again.\n")
        else:
            print("Sorry, you lose!\nI was thinking of '{}'.".format(word))
            break

NameError: name 'random' is not defined

In [21]:
import speech_recognition as  sr 
import  os
from pydub import AudioSegment
from  pydub.silence  import  split_on_silence
from pydub.playback import play

# a function that splits the audio file into chunks 
# and applies speech recognition 

def silence_based_conversion(path = ""): 

    # open the audio file stored in 
    # the local system as a wav file. 
    song = AudioSegment.from_mp3(path)[0:45*1000] 

    # open a file where we will concatenate 
    # and store the recognized text 
    fh = open("recognized.txt", "w+") 

    # split track where silence is 0.5 seconds 
    # or more and get chunks 
    chunks = split_on_silence(song, 
        # must be silent for at least 0.5 seconds 
        # or 500 ms. adjust this value based on user 
        # requirement. if the speaker stays silent for 
        # longer, increase this value. else, decrease it. 
        min_silence_len = 750, 

        # consider it silent if quieter than -16 dBFS 
        # adjust this per requirement 
        silence_thresh = -50
    ) 

    # create a directory to store the audio chunks. 
    try: 
        os.mkdir('audio_chunks') 
    except(FileExistsError): 
        pass

    # move into the directory to 
    # store the audio files. 
    os.chdir('audio_chunks') 

    i = 0
    # process each chunk 
    for chunk in chunks:	
        # Create 0.5 seconds silence chunk 
        chunk_silent = AudioSegment.silent(duration = 10) 

        # add 0.5 sec silence to beginning and 
        # end of audio chunk. This is done so that 
        # it doesn't seem abruptly sliced. 
        audio_chunk = chunk_silent + chunk + chunk_silent 

        # export audio chunk and save it in 
        # the current directory. 
        print("saving chunk{0}.wav".format(i)) 
        # specify the bitrate to be 192 k 
        audio_chunk.export("./chunk{0}.wav".format(i), bitrate ='192k', format ="wav") 

        # the name of the newly created chunk 
        filename = 'chunk'+str(i)+'.wav'

        print("Processing chunk "+str(i)) 

        # get the name of the newly created chunk 
        # in the AUDIO_FILE variable for later use. 
        file = filename 

        # create a speech recognition object 
        r = sr.Recognizer() 

        # recognize the chunk 
        with sr.AudioFile(file) as source: 
            # remove this if it is not working 
            # correctly. 
            # r.adjust_for_ambient_noise(source) 
            audio_listened = r.listen(source) 

        try: 
            # try converting it to text 
            rec = r.recognize_google(audio_listened) 
            # write the output to the file. 
            fh.write(rec+".") 

        # catch any errors. 
        except sr.UnknownValueError: 
            print("Could not understand audio") 

        except sr.RequestError as e: 
            print("Could not request results. check your internet connection") 

        i += 1

    os.chdir('..') 


if __name__ == '__main__': 

    print('Enter the audio file path') 

    path = input() 

    silence_based_conversion(path) 

Enter the audio file path


KeyboardInterrupt: 

In [ ]:
import speech_recognition as  sr 
import  os
from pydub import AudioSegment
from  pydub.silence  import  split_on_silence
from pydub.playback import play

# a function that splits the audio file into chunks 
# and applies speech recognition 

def silence_based_conversion(path = ""): 

	# open the audio file stored in 
	# the local system as a wav file. 
	song = AudioSegment.from_mp3(path)[0:45*1000] 

	# open a file where we will concatenate 
	# and store the recognized text 
	fh = open("recognized.txt", "w+") 
		
	# split track where silence is 0.5 seconds 
	# or more and get chunks 
	chunks = split_on_silence(song, 
		# must be silent for at least 0.5 seconds 
		# or 500 ms. adjust this value based on user 
		# requirement. if the speaker stays silent for 
		# longer, increase this value. else, decrease it. 
		min_silence_len = 750, 

		# consider it silent if quieter than -16 dBFS 
		# adjust this per requirement 
		silence_thresh = -50
	) 

	# create a directory to store the audio chunks. 
	try: 
		os.mkdir('audio_chunks') 
	except(FileExistsError): 
		pass

	# move into the directory to 
	# store the audio files. 
	os.chdir('audio_chunks') 

	i = 0
	# process each chunk 
	for chunk in chunks:	
		# Create 0.5 seconds silence chunk 
		chunk_silent = AudioSegment.silent(duration = 10) 

		# add 0.5 sec silence to beginning and 
		# end of audio chunk. This is done so that 
		# it doesn't seem abruptly sliced. 
		audio_chunk = chunk_silent + chunk + chunk_silent 

		# export audio chunk and save it in 
		# the current directory. 
		print("saving chunk{0}.wav".format(i)) 
		# specify the bitrate to be 192 k 
		audio_chunk.export("./chunk{0}.wav".format(i), bitrate ='192k', format ="wav") 

		# the name of the newly created chunk 
		filename = 'chunk'+str(i)+'.wav'

		print("Processing chunk "+str(i)) 

		# get the name of the newly created chunk 
		# in the AUDIO_FILE variable for later use. 
		file = filename 

		# create a speech recognition object 
		r = sr.Recognizer() 

		# recognize the chunk 
		with sr.AudioFile(file) as source: 
			# remove this if it is not working 
			# correctly. 
			# r.adjust_for_ambient_noise(source) 
			audio_listened = r.listen(source) 

		try: 
			# try converting it to text 
			rec = r.recognize_google(audio_listened) 
			# write the output to the file. 
			fh.write(rec+".") 

		# catch any errors. 
		except sr.UnknownValueError: 
			print("Could not understand audio") 

		except sr.RequestError as e: 
			print("Could not request results. check your internet connection") 

		i += 1

	os.chdir('..') 


if __name__ == '__main__': 
		
	print('Enter the audio file path') 

	path = input() 

	silence_based_conversion(path) 

In [ ]:
import speech_recognition as  sr 
import  os
from pydub import AudioSegment
from  pydub.silence  import  split_on_silence
from pydub.playback import play

# a function that splits the audio file into chunks 
# and applies speech recognition 

def silence_based_conversion(path = ""): 

    # open the audio file stored in 
    # the local system as a wav file. 
    song = AudioSegment.from_mp3(path)[0:45*1000] 

    # open a file where we will concatenate 
    # and store the recognized text 
    fh = open("recognized.txt", "w+") 

    # split track where silence is 0.5 seconds 
    # or more and get chunks 
    chunks = split_on_silence(song, 
        # must be silent for at least 0.5 seconds 
        # or 500 ms. adjust this value based on user 
        # requirement. if the speaker stays silent for 
        # longer, increase this value. else, decrease it. 
        min_silence_len = 750, 

        # consider it silent if quieter than -16 dBFS 
        # adjust this per requirement 
        silence_thresh = -50
    ) 

    # create a directory to store the audio chunks. 
    try: 
        os.mkdir('audio_chunks') 
    except(FileExistsError): 
        pass

    # move into the directory to 
    # store the audio files. 
    os.chdir('audio_chunks') 

    i = 0
    # process each chunk 
    for chunk in chunks:	
        # Create 0.5 seconds silence chunk 
        chunk_silent = AudioSegment.silent(duration = 10) 

        # add 0.5 sec silence to beginning and 
        # end of audio chunk. This is done so that 
        # it doesn't seem abruptly sliced. 
        audio_chunk = chunk_silent + chunk + chunk_silent 

        # export audio chunk and save it in 
        # the current directory. 
        print("saving chunk{0}.wav".format(i)) 
        # specify the bitrate to be 192 k 
        audio_chunk.export("./chunk{0}.wav".format(i), bitrate ='192k', format ="wav") 

        # the name of the newly created chunk 
        filename = 'chunk'+str(i)+'.wav'

        print("Processing chunk "+str(i)) 

        # get the name of the newly created chunk 
        # in the AUDIO_FILE variable for later use. 
        file = filename 

        # create a speech recognition object 
        r = sr.Recognizer() 

        # recognize the chunk 
        with sr.AudioFile(file) as source: 
            # remove this if it is not working 
            # correctly. 
            # r.adjust_for_ambient_noise(source) 
            audio_listened = r.listen(source) 

        try: 
            # try converting it to text 
            rec = r.recognize_google(audio_listened) 
            # write the output to the file. 
            fh.write(rec+".") 

        # catch any errors. 
        except sr.UnknownValueError: 
            print("Could not understand audio") 

        except sr.RequestError as e: 
            print("Could not request results. check your internet connection") 

        i += 1

    os.chdir('..') 


if __name__ == '__main__': 

    print('Enter the audio file path') 

    path = input() 

    silence_based_conversion(path) 